# Data Preprocessing and Training 

### Camera
For the guidance system, we used pre-recorded video feed of several controlled runs around a track. This provided the data that will be labelled using the `labeldata.py` script provided. 

### Load Data

In [2]:
import torchvision.transforms as transforms
from xy_dataset import XYDataset

TASK = 'road_following'

CATEGORIES = ['apex']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
dataset = XYDataset('lane_following', CATEGORIES, TRANSFORMS, random_hflip=True)

/home/snell/pynis/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: ''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


For the purpose of Data Collection, a separate module has been created that takes in pre-recorded video feed and allows for labelling through mouse detection. The following provided assumes that this module has been done prior. 

In [5]:
print(len(dataset))
print(dataset[0])

16553
(tensor([[[-0.3883, -0.4054, -0.6452,  ...,  1.8037,  1.8037,  1.8037],
         [-0.6794, -0.7822, -0.9705,  ...,  1.8037,  1.8037,  1.8037],
         [-1.3815, -1.5014, -1.5699,  ...,  1.8037,  1.8037,  1.8037],
         ...,
         [-0.4739, -0.5253, -0.4568,  ..., -0.9705, -1.0562, -0.9534],
         [-0.5253, -0.5082, -0.4911,  ..., -0.9705, -1.0219, -0.9020],
         [-0.5596, -0.5767, -0.5596,  ..., -1.0390, -1.0904, -1.0219]],

        [[-0.3200, -0.2850, -0.4776,  ...,  1.9734,  1.9734,  1.9734],
         [-0.4951, -0.5651, -0.7227,  ...,  1.9734,  1.9734,  1.9734],
         [-1.1078, -1.1954, -1.2654,  ...,  1.9734,  1.9734,  1.9734],
         ...,
         [-0.2325, -0.2500, -0.1800,  ..., -0.6001, -0.7227, -0.6352],
         [-0.2850, -0.2500, -0.2675,  ..., -0.6352, -0.6877, -0.5826],
         [-0.3550, -0.3725, -0.3725,  ..., -0.7052, -0.7402, -0.7052]],

        [[-0.0615, -0.0615, -0.2707,  ...,  2.1868,  2.1868,  2.1868],
         [-0.3055, -0.3753, -0.5321,  

### Model Setup

In [4]:
import torch
import torchvision

device = torch.device('cuda')
output_dim = 2 * len(dataset.categories)  # x, y coordinate for each category

#############################################
## Uncomment other models if desired
#############################################

# ALEXNET
# model = torchvision.models.alexnet(pretrained=True)
# model.classifier[-1] = torch.nn.Linear(4096, output_dim)

# SQUEEZENET 
# model = torchvision.models.squeezenet1_1(pretrained=True)
# model.classifier[1] = torch.nn.Conv2d(512, output_dim, kernel_size=1)
# model.num_classes = len(dataset.categories)

# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)

# RESNET 34
# model = torchvision.models.resnet34(pretrained=True)
# model.fc = torch.nn.Linear(512, output_dim)

# DENSENET 121
# model = torchvision.models.densenet121(pretrained=True)
# model.classifier = torch.nn.Linear(model.num_features, output_dim)

model = model.to(device)

def load_model(c):
    model.load_state_dict(torch.load(model_path_widget.value))

    
def save_model(c, save_path="Car_Model.pth"):
    torch.save(model.state_dict(), save_path)

/home/snell/pynis/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/snell/pynis/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Training

In [5]:
BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())

def train_eval(is_training, epochs_value=1):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget
    
    train_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True
    )

    time.sleep(1)

    if is_training:
        model = model.train()
    else:
        model = model.eval()

    while epochs_value > 0:
        i = 0
        sum_loss = 0.0
        error_count = 0.0
        for images, category_idx, xy in iter(train_loader):
            # send data to device
            images = images.to(device)
            xy = xy.to(device)

            if is_training:
                # zero gradients of parameters
                optimizer.zero_grad()

            # execute model to get outputs
            outputs = model(images)

            # compute MSE loss over x, y coordinates for associated categories
            loss = 0.0
            for batch_idx, cat_idx in enumerate(list(category_idx.flatten())):
                loss += torch.mean((outputs[batch_idx][2 * cat_idx:2 * cat_idx+2] - xy[batch_idx])**2)
            loss /= len(category_idx)

            if is_training:
                # run backpropogation to accumulate gradients
                loss.backward()

                # step optimizer to adjust parameters
                optimizer.step()

            # increment progress
            count = len(category_idx.flatten())
            i += count
            sum_loss += float(loss)
            epochs_value -= 1
            
        if is_training:
            pass
        else:
            break

    

With the training function setup, run the following to complete the training process. 

In [6]:
import time
start = time.time()
train_eval(True, 8)
print(f"Elapsed time: {time.time() - start}")
start = time.time()
print("Saving model...")
save_model(model, "lane_following_model.pth")
print(f"Model saved!\nElapsed time: {time.time() - start}\nDONE")

Elapsed time: 5595.091841936111
Saving model...
Model saved!
Elapsed time: 0.49799370765686035
DONE
